Code from A3 problem1-prompt.ipynb and https://github.com/SewoongLee/reproduce-llama3-arithmetic/blob/main/llama3-tutorial-gsm8k.ipynb

You need to set your OPENAI_API_KEY environment variable in .env

In [4]:
from openai import OpenAI
from tqdm import tqdm
import textwrap
import dotenv
import os
%load_ext dotenv
%dotenv

client=OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# we use gpt-4o-mini as default: current pricing is 
# $0.150 / 1M input tokens, $0.600 / 1M output tokens
def generate_gpt_response(prompt,messages=[], model="gpt-4o-mini", temperature=0.7):
    messages.append({"role": "user", "content": prompt})
            
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature = temperature
    )
    content=response.choices[0].message.content
    messages.append({'role': 'assistant', 'content': content})
    return messages


def print_message(message, width=80):
    text=message["role"]+": " + message["content"]
    wrapped_text = textwrap.fill(text, width=width)
    
    for line in wrapped_text.split('\n'):
        print(f" {line.ljust(width)} ")        
    

cannot find .env file


load ablated gsm8k json files

In [9]:
import os
import random
import json

ablated_dataset = []
folder_path = "gsm_template/output"
for file_name in os.listdir(folder_path):
    f_path = os.path.join(folder_path, file_name)

    with open(f_path, 'r') as f:
        lines = f.readlines()

    random_lines = random.sample([json.loads(item.strip()) for item in lines], min(10, len(lines)))
    print(random_lines)
    print()

    ablated_dataset.append(random_lines)


[{'problem': 'Elvis Hammer sold clips to 48 of their friends in April, and then they sold 50% as many clips in May.How many clips did Elvis Hammer sell altogether in April and May?', 'result': '72', 'template_name': 0, 'idx': 64}, {'problem': 'German Redondo sold clips to 48 of their friends in April, and then they sold 50% as many clips in May.How many clips did German Redondo sell altogether in April and May?', 'result': '72', 'template_name': 0, 'idx': 88}, {'problem': 'Marilu Laing sold clips to 48 of their friends in April, and then they sold 50% as many clips in May.How many clips did Marilu Laing sell altogether in April and May?', 'result': '72', 'template_name': 0, 'idx': 95}, {'problem': 'Jeremiah Unruh sold clips to 48 of their friends in April, and then they sold 50% as many clips in May.How many clips did Jeremiah Unruh sell altogether in April and May?', 'result': '72', 'template_name': 0, 'idx': 44}, {'problem': 'Elliot Juares sold clips to 48 of their friends in April, 

We will use the following system prompt

In [5]:
system_prompt= \
'''You are an expert math tutor specializing in grade-school-level problems like those in the GSM8K dataset. 
Solve each problem step by step, explaining the reasoning and calculations clearly and concisely to help the student understand the process. 
Please return your final answer after the delimiter \"####\" as a numeric number'''

We can answer from gpt api using fewshots in context learning and system prompt

In [ ]:
import re

def extract_ans_from_response(answer: str, eos=None):
    if eos:
        answer = answer.split(eos)[0].strip()

    answer = answer.split('####')[-1].strip()

    for remove_char in [',', '$', '%', 'g']:
        answer = answer.replace(remove_char, '')

    try:
        return int(answer)
    except ValueError:
        return answer

def get_num_answer(text):
    num_matches = re.findall(r'\d+\.?\d*', text)

    if not num_matches:
        return None
    
    final_answer = num_matches[-1]

    return final_answer

# Implement the function answer_from_gpt(question,temperature,num_shots), which should return the gpt answer as a string
# (see how it is called in the function evaluate_fewshots below)
# temperature is the temperature parameter passed to gpt api
# if num_shots>0, you should use the first num_shots examples in train_data for in-context learning
# Please use the system prompt provided above (see lecture10-gpt.ipynb for examples of system prompt usage)
#
def answer_from_gpt(question, temperature, num_shots):
    example = []

    # if num_shots > 0:
    #     for i in range(num_shots):
    #         curr_ex = train_data[i]
    #         example.append({"role": "user", "content": curr_ex['question']})
    #         example.append({"role": "assistant", "content": curr_ex['answer']})

    full_prompt = system_prompt + "\n" + question
    # print("Q: ", full_prompt)

    messages = generate_gpt_response(prompt=full_prompt, messages=example, temperature=temperature)
    answer = messages[-1]["content"]
    # print("A: ", answer)
    last_token = extract_ans_from_response(answer)
    return last_token

Evaluate on a small subset of gsm8k test data

In [ ]:
def evaluate_fewshots(data, temperature,num_shots):
    num_corrects=0
    for k in  tqdm(range(len(data))):
        gpt_answer=answer_from_gpt(data[k]['problem'],temperature=temperature,num_shots=num_shots)
        true_answer= data[k]['result']
        print(f'k={k}: gpt_answer={gpt_answer}  true_answer={true_answer}')   # this is for debug only
        if float(gpt_answer) == float(true_answer):
            num_corrects += 1
    print(f'accuracy={num_corrects/len(data):0.2f}')

temperature = 0
num_shots = 0

print(f'=== temperature={temperature} num_shots={num_shots} ===')
for i in range(len(ablated_dataset)):
    curr_ablation = ablated_dataset[i]

    print("Question ", i)
    evaluate_fewshots(curr_ablation, temperature,num_shots)
    print()

=== temperature=0 num_shots=0 ===


 10%|█         | 1/10 [00:03<00:27,  3.09s/it]

k=0: gpt_answer=72  true_answer=72


 20%|██        | 2/10 [00:06<00:27,  3.44s/it]

k=1: gpt_answer=72  true_answer=72


...

k=8: gpt_answer=35  true_answer=35


100%|██████████| 10/10 [01:15<00:00,  7.54s/it]


k=9: gpt_answer=35  true_answer=35
accuracy=1.00



 10%|█         | 1/10 [00:03<00:35,  3.98s/it]

k=0: gpt_answer=48  true_answer=48
